In [31]:
import openai

# configure the client's secret key
with open('secret-key.txt', 'r') as file:
    secret_key = file.read().strip()
openai.api_key = secret_key

max_tokens = 4092

In [32]:
import time
def prompt(question):
    start_time_seconds = time.time()
    print(f"asking chatgpt: {question}")
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo", #"text-davinci-003",
        messages=[
            {"role": "system", "content": "You are a helpful assistant that answers questions accurately, without making up facts."},
            {"role": "user", "content": question}
        ],
    )
    # print(response)
    choices = response.choices
    choice = choices[0]
    message = choice['message']
    answer = message['content']
    print(answer)
    print(f"answer received in {time.time() - start_time_seconds} seconds.")

def stream_prompt(question):
    start_time_seconds = time.time()
    print(f"asking chatgpt: {question}")
    stream = openai.ChatCompletion.create(
        model="gpt-3.5-turbo", #"text-davinci-003",
        messages=[
            {"role": "system", "content": "You are a helpful assistant that answers questions accurately, without making up facts."},
            {"role": "user", "content": question}
        ],
        stream=True,
    )
    for output in stream:
        choices = output['choices']
        choice = choices[0]
        delta = choice['delta']
        if 'content' in delta:
            print(delta['content'], end='')
    print()
    print(f"answer received in {time.time() - start_time_seconds} seconds.")

In [33]:
stream_prompt("""
What is a camel?
""")

asking chatgpt: 
What is a camel?

A camel is a large mammal with a humped back, long legs, and a long neck. It is native to the deserts of Africa and Asia and is well adapted to living in hot and arid conditions. Camels are known for their ability to store water in their humps, enabling them to survive in environments with limited water sources. They are often used as pack animals and are also raised for their milk, meat, and hair.
answer received in 10.847567319869995 seconds.


In [39]:
# https://github.com/openai/openai-cookbook/blob/main/examples/How_to_call_functions_for_knowledge_retrieval.ipynb
import json

def get_function_calls_details_from_llm_response(response):
    response_message = response.choices[0].message
    tool_calls = response_message.get('tool_calls')
    results = None
    # Step 2: check if the model wanted to call a function
    if tool_calls:
        results = []
        for tool_call in tool_calls:
            function_name = tool_call.function.name
            arguments = tool_call.function.arguments
            result = {
                "function_name": function_name,
                "arguments": arguments,
                # data needed for subsequent calls to chatgpt
                "metadata": {
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "name": function_name,
                    "content": None, # todo: populate this once the function has been executed.
                },
            }
            results.append(result)
    return results

def function_call_prompt(question):
    tools = [
        {
            "type": "function",
            "function": {
                "name": "get_user_details",
                "description": "Retrieves information for a user based on the combination of their first and last names.  It returns information about the user's age, location, and profession.",
                "parameters":{
                    "type": "object",
                    "properties": {
                        "first_name": {
                            "type": "string",
                            "description": "First name of the user to get details of."
                        },
                        "last_name": {
                            "type": "string",
                            "description": "Last name of the user to get details of."
                        }
                    },
                    "required": ["first_name", "last_name"]
                }
            },

        },
    ]

    start_time_seconds = time.time()
    print(f"asking chatgpt: {question}")
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant that answers questions accurately, without making up facts."},
            {"role": "user", "content": question}
        ],
        tools=tools
    )
    print(response)
    functions = get_function_calls_details_from_llm_response(response)
    if functions is not None:
        for function in functions:
            if function['function_name'] == 'get_user_details':
                content = get_user_details(function['arguments'])
                function['metadata']['content'] = content

    print(f"answer received in {time.time() - start_time_seconds} seconds.")

# function that will be called, based on the LLM's response, which will indicate that it wants the function to be called, along with the arguments
# arguments are a json string.  e.g. '{"first_name": "Jason", "last_name: "McAffee"}'
def get_user_details(args):
    arguments = json.loads(args)
    print(f"get_user_details function called with arguments first_name: {arguments['first_name']}, last_name: {arguments['last_name']}")
    response = {
        "age": 44,
        "location": {
            "state": "Utah"
        },
        "profession": "Software Engineer"
    }
    return response

In [40]:
function_call_prompt("""
Who is user Jason McAffee?
""")

asking chatgpt: 
Who is user Jason McAffee?

{
  "id": "chatcmpl-8OdsAEvqPHUWMlLYShPRlc90SizCy",
  "object": "chat.completion",
  "created": 1700883834,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": null,
        "tool_calls": [
          {
            "id": "call_HnsHtNPVKC6rVYXYEG2duSs3",
            "type": "function",
            "function": {
              "name": "get_user_details",
              "arguments": "{\n  \"first_name\": \"Jason\",\n  \"last_name\": \"McAffee\"\n}"
            }
          }
        ]
      },
      "finish_reason": "tool_calls"
    }
  ],
  "usage": {
    "prompt_tokens": 123,
    "completion_tokens": 27,
    "total_tokens": 150
  }
}
get_user_details function called with arguments first_name: Jason, last_name: McAffee
answer received in 3.895408868789673 seconds.


In [41]:
function_call_prompt("""
What is a camel?
""")

asking chatgpt: 
What is a camel?

{
  "id": "chatcmpl-8OdteL587sxQQR10wCzkYh1u17ijP",
  "object": "chat.completion",
  "created": 1700883926,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "A camel is a large mammal with a hump on its back. It is known for its ability to survive in harsh desert environments due to its ability to store water. Camels are commonly used as pack animals and can travel long distances without needing much water. They are also known for their distinctive long necks and eyelashes."
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 120,
    "completion_tokens": 68,
    "total_tokens": 188
  }
}
answer received in 9.581519842147827 seconds.
